In [2]:
import pandas as pd
import os
from fpdf import FPDF
from PIL import Image

# 1. 데이터 로딩 (윈도우 경로)
df = pd.read_excel(r"C:\Users\yu\Desktop\test3\id.xlsx")

if '유형' not in df.columns:
    raise ValueError("엑셀 파일에 '유형' 열이 없습니다. H열에 '지문' 또는 '문제'가 있어야 합니다.")

# === 이미지를 지정 높이만큼 조각내 리스트로 반환 ===
def split_image_to_parts(img_path, part_height_px):
    parts = []
    with Image.open(img_path) as img:
        w, h = img.size
        for top in range(0, h, part_height_px):
            box = (0, top, w, min(top + part_height_px, h))
            part = img.crop(box)
            temp_path = os.path.join(r"C:\Users\yu\Desktop\test3", f"temp_part_{os.path.basename(img_path)}_{top}.png")
            part.save(temp_path)
            parts.append(temp_path)
    return parts

# 2. 문제 선택 함수 (난이도 기준)
def select_questions_by_level(level, num_questions):
    questions_df = df[(df['유형'] == '문제') & (df['난이도'] == level)]
    questions_df = questions_df.sample(frac=1).reset_index(drop=True)
    selected_questions = questions_df.head(num_questions)
    return selected_questions

def find_passage_for_question(question_row):
    pid = question_row['지문id']
    passage_row = df[(df['유형'] == '지문') & (df['지문id'] == pid)]
    if not passage_row.empty:
        return pid
    return None

def mm_to_px(mm, dpi=300):
    return int(mm * dpi / 25.4)

def insert_problems_with_dynamic_layout(pdf, problem_image_paths, max_page_height_mm=297, margin_mm=15, dpi=300, image_width_mm=90):
    max_page_height_px = mm_to_px(max_page_height_mm - 2 * margin_mm, dpi)
    i = 0
    n = len(problem_image_paths)
    while i < n:
        pdf.add_page()
        pdf.set_font("Arial", "B", 16)
        pdf.cell(0, 10, "문제", ln=True, align='C')
        y_offset = 20  # 시작 y 위치
        count_in_page = 0
        while i < n and count_in_page < 3:
            img_path = problem_image_paths[i]
            with Image.open(img_path) as img:
                w_px, h_px = img.size
                aspect_ratio = w_px / h_px
                img_display_width = image_width_mm
                img_display_height = img_display_width / aspect_ratio
                img_display_height_px = mm_to_px(img_display_height, dpi)
            # 높이에 따라 배치 개수 조정
            if img_display_height_px > max_page_height_px * 2 / 3:
                max_problems_in_page = 1
            elif img_display_height_px > max_page_height_px / 3:
                max_problems_in_page = 2
            else:
                max_problems_in_page = 3
            if count_in_page >= max_problems_in_page:
                break
            x = margin_mm
            y = y_offset
            pdf.image(img_path, x=x, y=y, w=img_display_width, h=img_display_height)
            y_offset += img_display_height + 10
            count_in_page += 1
            i += 1
        # 페이지 번호
        pdf.set_y(max_page_height_mm - margin_mm)
        pdf.set_font("Arial", "", 10)
        pdf.cell(0, 10, f"Page {pdf.page_no()}", align='C')
    return

# === [새로 추가] 지문을 한 페이지의 왼쪽/오른쪽 단에 배치 ===
def insert_passage_in_two_columns(pdf, passage_parts, image_width=90, max_image_height=250):
    page_width = 210
    page_height = 297
    column_gap = 10
    margin_x = 10
    margin_y = 10

    i = 0
    while i < len(passage_parts):
        pdf.add_page()
        current_y_left = margin_y
        current_y_right = margin_y

        # 왼쪽 단
        if i < len(passage_parts):
            with Image.open(passage_parts[i]) as img:
                img_width, img_height = img.size
                aspect_ratio = img_width / img_height
                img_new_height = image_width / aspect_ratio
                if img_new_height > max_image_height:
                    img_new_height = max_image_height
                img_new_width = image_width
            pdf.image(passage_parts[i], x=margin_x, y=current_y_left, w=img_new_width)
            current_y_left += img_new_height + 10
            i += 1

        # 오른쪽 단
        if i < len(passage_parts):
            with Image.open(passage_parts[i]) as img:
                img_width, img_height = img.size
                aspect_ratio = img_width / img_height
                img_new_height = image_width / aspect_ratio
                if img_new_height > max_image_height:
                    img_new_height = max_image_height
                img_new_width = image_width
            pdf.image(passage_parts[i], x=margin_x + image_width + column_gap, y=current_y_right, w=img_new_width)
            current_y_right += img_new_height + 10
            i += 1

# === 문제 2단 배치 함수(기존) ===
def insert_images_in_two_columns(pdf, image_paths, image_width=90, max_image_height=250):
    page_height = 297
    column_gap = 10
    margin_x = 10
    margin_y = 10
    column_width = image_width

    # 이미지가 1장뿐이면 그냥 한 쪽에만 출력
    if len(image_paths) == 1:
        pdf.add_page()
        with Image.open(image_paths[0]) as img:
            w, h = img.size
            aspect_ratio = w / h
            img_new_height = image_width / aspect_ratio
            if img_new_height > max_image_height:
                img_new_height = max_image_height
            img_new_width = image_width
        pdf.image(image_paths[0], x=margin_x, y=margin_y, w=img_new_width)
        return

    # 여러 장인 경우 2단으로 배치
    pdf.add_page()
    current_y_left = margin_y
    current_y_right = margin_y
    left = True  # 좌우 교대

    for image_path in image_paths:
        with Image.open(image_path) as img:
            w, h = img.size
            aspect_ratio = w / h
            img_new_height = image_width / aspect_ratio
            if img_new_height > max_image_height:
                img_new_height = max_image_height
            img_new_width = image_width

        if left:
            if current_y_left + img_new_height > page_height - margin_y:
                pdf.add_page()
                current_y_left = margin_y
                current_y_right = margin_y
            pdf.image(image_path, x=margin_x, y=current_y_left, w=img_new_width)
            current_y_left += img_new_height + 10
        else:
            if current_y_right + img_new_height > page_height - margin_y:
                pdf.add_page()
                current_y_left = margin_y
                current_y_right = margin_y
            pdf.image(image_path, x=margin_x + column_width + column_gap, y=current_y_right, w=img_new_width)
            current_y_right += img_new_height + 10

        left = not left

# === PDF 생성 함수 ===
def create_pdf_from_images(
    selected_questions,
    output_path=r"C:\Users\yu\Desktop\test3\시험지.pdf",
    max_passage_part_height=3500
):
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    temp_files = []

    for _, qrow in selected_questions.iterrows():
        # 1. 지문 처리
        passage_raw = qrow['지문id']
        passage_id = passage_raw.strftime('%Y-%m-%d') if isinstance(passage_raw, pd.Timestamp) else str(passage_raw)
        passage_path = os.path.join(r"C:\Users\yu\Desktop\test3\images\passages", f"{passage_id}.png")

        if os.path.exists(passage_path):
            parts = split_image_to_parts(passage_path, max_passage_part_height)
            insert_passage_in_two_columns(pdf, parts, image_width=90, max_image_height=250)
            temp_files.extend(parts)
        else:
            print(f"[지문 누락] {passage_path}")

        # 2. 문제 처리
        question_id = str(qrow['문제id'])
        question_path = os.path.join(r"C:\Users\yu\Desktop\test3\images\questions", f"{question_id}.png")

        if os.path.exists(question_path):
            insert_images_in_two_columns(pdf, [question_path], image_width=90, max_image_height=250)
        else:
            print(f"[문제 누락] {question_path}")

    pdf.output(output_path)
    print(f"✅ 시험지 PDF 저장 완료: {output_path}")

    # 임시파일 삭제
    for f in temp_files:
        try:
            os.remove(f)
        except:
            pass

# 1. 원하는 난이도의 문제 선택 (예: '중' 난이도에서 10문제)
selected_questions = select_questions_by_level(level='중', num_questions=10)

# 2. PDF 생성 (지문은 한 페이지에 2단, 3500픽셀 단위로 자동 분할)
create_pdf_from_images(
    selected_questions,
    output_path=r"C:\Users\yu\Desktop\test3\시험지.pdf",
    max_passage_part_height=3500
)


✅ 시험지 PDF 저장 완료: C:\Users\yu\Desktop\test3\시험지.pdf
